In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def calcular_taxa_notificacoes(arquivo_csv, arquivo_populacao):
    df = pd.read_csv(arquivo_csv)
    df_populacao = pd.read_csv(arquivo_populacao)
    
    df['dataNotificacao'] = pd.to_datetime(df['dataNotificacao'], errors='coerce')
    df = df.dropna(subset=['dataNotificacao'])
    df = df[df['dataNotificacao'].dt.year > 2021]
    
    df['ano_mes'] = df['dataNotificacao'].dt.to_period('M').astype(str)
    
    # Agrupar por município e mês
    notificacoes_por_municipio = df.groupby(['municipioNotificacao', 'ano_mes']).size().reset_index(name='quantidade')
    
    # Associar população a cada município
    df_populacao.rename(columns={"Município": "municipioNotificacao", "População": "populacao"}, inplace=True)
    notificacoes_por_municipio = notificacoes_por_municipio.merge(df_populacao, on="municipioNotificacao", how="left")
    
    # Calcular taxa por 100.000 habitantes
    notificacoes_por_municipio['taxa_100k'] = (notificacoes_por_municipio['quantidade'] / notificacoes_por_municipio['populacao']) * 100000
    notificacoes_por_municipio.dropna(inplace=True)  # Remover cidades sem dados de população
    
    return notificacoes_por_municipio

def plotar_top_10_municipios(notificacoes_por_municipio):
    top_10 = notificacoes_por_municipio.groupby("municipioNotificacao")["quantidade"].sum().nlargest(10)
    
    top_10_df = notificacoes_por_municipio[notificacoes_por_municipio["municipioNotificacao"].isin(top_10.index)]
    pivot_df = top_10_df.pivot(index="ano_mes", columns="municipioNotificacao", values="quantidade").fillna(0)
    
    pivot_df.plot(kind='bar', stacked=True, figsize=(12, 6))
    plt.xticks(rotation=45)
    plt.xlabel("Ano-Mês")
    plt.ylabel("Número de Notificações")
    plt.title("Top 10 Municípios com Maior Número de Notificações")
    plt.legend(title="Municípios")
    plt.grid()
    plt.show()

def prever_proximo_mes(notificacoes_por_mes):
    notificacoes_por_mes['ano_mes'] = pd.to_datetime(notificacoes_por_mes['ano_mes'].astype(str))
    notificacoes_por_mes['timestamp'] = (notificacoes_por_mes['ano_mes'] - notificacoes_por_mes['ano_mes'].min()).dt.days
    
    X = notificacoes_por_mes[['timestamp']]
    y = notificacoes_por_mes['quantidade']
    
    # Criando um modelo de regressão polinomial de grau 2
    modelo = make_pipeline(PolynomialFeatures(2), LinearRegression())
    modelo.fit(X, y)
    
    proximo_mes = np.array([[notificacoes_por_mes['timestamp'].max() + 30]])
    previsao = modelo.predict(proximo_mes)
    
    # Não permitir que a previsão seja negativa
    previsao = max(0, previsao[0])
    
    print("Previsão para o próximo mês:", previsao)
    return previsao

def prever_top_10_municipios(notificacoes_por_municipio):
    top_10 = notificacoes_por_municipio.groupby("municipioNotificacao")["quantidade"].sum().nlargest(10).index
    previsoes = {}
    
    for municipio in top_10:
        subset = notificacoes_por_municipio[notificacoes_por_municipio["municipioNotificacao"] == municipio]
        previsoes[municipio] = prever_proximo_mes(subset)
    
    return previsoes

# Sugestões de outros modelos:
# - Regressão Linear Simples: Fácil de interpretar, mas pode não capturar padrões complexos.
# - ARIMA: Boa para séries temporais, mas requer ajustes de parâmetros.
# - Redes Neurais (LSTM/RNN): Captura padrões complexos, mas exige mais dados e poder computacional.

# Chamada da função (substituir 'dados_completos.csv' e 'municipios_habitantes_mg.csv' pelo nome correto dos arquivos CSV)
# df_resultado = calcular_taxa_notificacoes('dados_completos.csv', 'municipios_habitantes_mg.csv')
# plotar_top_10_municipios(df_resultado)
# previsoes_top_10 = prever_top_10_municipios(df_resultado)

/home/tonykaku/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
df_resultado = calcular_taxa_notificacoes('dados_completos.csv', 'municipios_habitantes_mg.csv')

/tmp/ipykernel_3672/300807083.py:9: DtypeWarning: Columns (12,16,21,23,26,31,33,34,37,57,58,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo_csv)


In [ ]:
plotar_top_10_municipios(df_resultado)


In [ ]:
previsoes_top_10 = prever_top_10_municipios(df_resultado)


In [ ]:
previsoes_top_10